<a href="https://colab.research.google.com/github/VaishnavSubramanian/voiceTreatment/blob/main/after.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install faster-whisper

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 131.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.0 MB/s eta 0:00:00


In [ ]:
pip install noisereduce

In [ ]:
import librosa, soundfile as sf
import noisereduce as nr
import numpy as np
from pydub import AudioSegment, silence

# 1️⃣ Load audio
audio = AudioSegment.from_file("my_lazada_sample1.wav")
audio = audio.set_channels(1).set_frame_rate(16000)

# 2️⃣ Remove long silences (>800ms below -35dB)
nonsilent = silence.split_on_silence(
    audio,
    min_silence_len=800,
    silence_thresh=audio.dBFS - 35,
    keep_silence=200  # keep some padding
)
cleaned_audio = AudioSegment.silent(duration=0)
for chunk in nonsilent:
    cleaned_audio += chunk
cleaned_audio.export("temp_nosilence.wav", format="wav")

# 3️⃣ Noise reduction (spectral gating)
y, sr = librosa.load("temp_nosilence.wav", sr=16000)
reduced_noise = nr.reduce_noise(y=y, sr=sr,
                                 prop_decrease=1.0,       # Amount of noise reduction (0.0–1.0)
    stationary=True,         # True for constant background noise
    use_tqdm=False,          # Show progress bar
    n_fft=1024,              # FFT window size
    win_length=1024,         # Window length
    hop_length=256           # Hop size
          )

sf.write("sample_denoised.wav", reduced_noise, sr)
print("✅ Saved denoised, silence-removed audio as 'sample_denoised.wav'")

/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


✅ Saved denoised, silence-removed audio as 'sample_denoised.wav'


In [ ]:
from faster_whisper import WhisperModel

model = WhisperModel("medium", device="cpu", compute_type="int8")

segments, info = model.transcribe("sample_denoised.wav", language='en')
for segment in segments:
    print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocabulary.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.bin:   0%|          | 0.00/1.53G [00:00<?, ?B/s]

[0.00s -> 5.60s]  Hi, thank you for contacting Lata Reyes, hello?
[6.60s -> 8.00s]  Can you give my order number?
[8.90s -> 9.50s]  Yes, sir
[10.90s -> 11.80s]  492
[12.60s -> 13.10s]  Okay
[15.20s -> 16.00s]  619
[18.40s -> 18.80s]  Okay
[20.10s -> 21.00s]  870
[25.70s -> 27.20s]  And then Mr. Herbert Lai
[28.40s -> 29.30s]  Yes, correct
[30.00s -> 30.50s]  Yes, sir
[34.50s -> 37.30s]  Keep regarding your luncheon meat combo
[38.60s -> 39.20s]  Alright
[39.90s -> 43.60s]  It's more than 72 hours now, it's still down to the top
[44.80s -> 45.30s]  So
[46.40s -> 48.30s]  So what is the status now?
[49.00s -> 51.70s]  I mean, I cannot be with you for so long
[53.40s -> 55.00s]  This food is still in progress
[55.50s -> 56.70s]  Yeah, but it's too long
[57.70s -> 61.10s]  I mean, it says to please give us two business working days
[61.10s -> 63.10s]  It's a bit more than three days more
[65.50s -> 67.90s]  Still in progress checking the issue, sir
[68.10s -> 71.70s]  But this food in prog

In [ ]:
from faster_whisper import WhisperModel

model = WhisperModel("large", device="cuda", compute_type="int8")

segments, info = model.transcribe("sample_denoised.wav", language='en')
for segment in segments:
    print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))

model.bin:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

[0.00s -> 5.44s]  Hi, thank you for contacting Azad Aya, hello.
[6.56s -> 7.80s]  Can you give my order number?
[8.82s -> 9.46s]  Yes, sir.
[11.04s -> 21.00s]  492-619-878.
[25.26s -> 27.16s]  And then Mr. Herbert Lai.
[28.28s -> 29.00s]  Yes, correct.
[30.00s -> 37.10s]  What is the case regarding your luncheon meat combo?
[38.62s -> 39.14s]  Alright.
[39.98s -> 42.06s]  It's more than 72 hours now.
[42.20s -> 43.22s]  It's still not resolved.
[44.46s -> 48.18s]  So, what is the status now?
[48.76s -> 51.58s]  I mean, I cannot be voting for so long.
[53.30s -> 55.04s]  Dispute is still in progress, sir.
[55.46s -> 56.62s]  Yeah, but it's too long.
[56.62s -> 59.12s]  I mean, it says that...
[59.12s -> 59.98s]  Please give us...
[59.98s -> 61.18s]  Two business hooking days.
[61.36s -> 63.14s]  It's a little bit more than three days more.
[65.56s -> 67.96s]  Still in progress checking the issue, sir.
[68.36s -> 71.78s]  But dispute in progress as of 5 November.
[72.68s -> 73.82s]  Yeah

In [ ]:
from google.colab import drive
drive.mount('/content/drive')